In [2]:
#!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.7 MB/s eta 0:00:00


In [4]:
!pip install qiskit-aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 92.2 MB/s eta 0:00:00


In [9]:
from qiskit_aer import Aer
import numpy as np

def quantum_swap_test(psi, phi, shots=1024):
    n = len(psi)
    qc = QuantumCircuit(1 + 2 * n, 1)

    qc.h(0)


    for i, angle in enumerate(psi):
        qc.ry(angle, i + 1)
    for i, angle in enumerate(phi):
        qc.ry(angle, i + n + 1)

    for i in range(n):
        qc.cswap(0, i + 1, i + n + 1)

    qc.h(0)
    qc.measure(0, 0)

    backend = Aer.get_backend('qasm_simulator')
    job = backend.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    prob_0 = counts.get('0', 0) / shots
    similarity = 2 * prob_0 - 1  # Probability difference

    return similarity, counts


In [10]:
psi = [np.pi / 2, np.pi / 4]
phi = [np.pi / 3, np.pi / 6]

similarity, counts = quantum_swap_test(psi, phi)
print("Similarity:", similarity)
print("Measurement counts:", counts)

Similarity: 0.919921875
Measurement counts: {'1': 41, '0': 983}


In [11]:
import torch

similarities = torch.tensor([0.9199, 0.8, 0.5])
attention_weights = torch.softmax(similarities, dim=0)
print(attention_weights)

tensor([0.3931, 0.3487, 0.2583])


In [12]:
import torch
import torch.nn as nn
from qiskit import QuantumCircuit
from qiskit_aer import Aer
import numpy as np

class QuantumAttention(nn.Module):
    def __init__(self, embedding_dim, shots=512):
        super(QuantumAttention, self).__init__()
        self.embedding_dim = embedding_dim
        self.shots = shots
        self.backend = Aer.get_backend('qasm_simulator')

    def quantum_similarity(self, vec_a, vec_b):
        qc = QuantumCircuit(1 + 2 * self.embedding_dim, 1)

        qc.h(0)

        for i, angle in enumerate(vec_a):
            qc.ry(angle, i + 1)
        for i, angle in enumerate(vec_b):
            qc.ry(angle, i + self.embedding_dim + 1)

        for i in range(self.embedding_dim):
            qc.cswap(0, i + 1, i + self.embedding_dim + 1)

        qc.h(0)
        qc.measure(0, 0)

        result = self.backend.run(qc, shots=self.shots).result()
        counts = result.get_counts()
        prob_0 = counts.get('0', 0) / self.shots
        similarity = 2 * prob_0 - 1
        return similarity

    def forward(self, queries, keys, values):
        batch_size, seq_len, _ = queries.size()
        attn_scores = torch.zeros(batch_size, seq_len, seq_len)

        queries_np = queries.detach().numpy()
        keys_np = keys.detach().numpy()

        for b in range(batch_size):
            for i in range(seq_len):
                for j in range(seq_len):
                    sim = self.quantum_similarity(queries_np[b, i], keys_np[b, j])
                    attn_scores[b, i, j] = sim

        attn_weights = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_weights, values)
        return output

    def quantum_similarity(self, vec_a, vec_b):

        vec_a = np.clip(vec_a, -1, 1) * np.pi
        vec_b = np.clip(vec_b, -1, 1) * np.pi

        qc = QuantumCircuit(1 + 2 * len(vec_a), 1)
        qc.h(0)

        for i, angle in enumerate(vec_a):
            qc.ry(angle, i + 1)
        for i, angle in enumerate(vec_b):
            qc.ry(angle, i + len(vec_a) + 1)

        for i in range(len(vec_a)):
            qc.cswap(0, i + 1, i + len(vec_a) + 1)

        qc.h(0)
        qc.measure(0, 0)

        result = self.backend.run(qc, shots=self.shots).result()
        counts = result.get_counts()
        prob_0 = counts.get('0', 0) / self.shots
        similarity = 2 * prob_0 - 1
        return similarity

class TransformerClassic(nn.Module):
    def __init__(self, embedding_dim=128, num_heads=4):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

    def forward(self, x):
        x = x.permute(1, 0, 2)
        attn_output, _ = self.attention(x, x, x)
        return attn_output.permute(1, 0, 2)


class TransformerQuantum(nn.Module):
    def __init__(self, embedding_dim=4, shots=256):
        super().__init__()
        self.attention = QuantumAttention(embedding_dim=embedding_dim, shots=shots)

    def forward(self, x):
        return self.attention(x, x, x)



In [16]:
batch_size = 2
seq_len = 3
embedding_dim_classic = 128
embedding_dim_quantum = 4

x_classic = torch.rand(batch_size, seq_len := 10, 128)

x_quantum = torch.rand(batch_size, seq_len, 4) * 2 - 1


In [18]:
import time


model_classic = TransformerClassic(embedding_dim=embedding_dim_classic)
start = time.time()
output_classic = model_classic(x_classic)
time_classic = time.time() - start

# Quantum Attention Timing
model_quantum = TransformerQuantum(embedding_dim=embedding_dim_quantum, shots=256)
start = time.time()
output_quantum = model_quantum(x_quantum)
time_quantum = time.time() - start

print(f"Classical Attention Time: {time_classic:.2f}s")
print(f"Quantum Attention Time: {time_quantum:.2f}s")


Classical Attention Time: 0.00s
Quantum Attention Time: 0.60s
